In [ ]:
#Importação das bibliotecas e módulos necessários
import pandas as pd
import plotly.express as px
import datetime as dt
import os
from google.colab import drive

#Realização do mount do drive para conseguir utilizar os datasets
drive.mount('/content/drive')

#Abertura do dataset do INMET para obter os valores de latitude e longitude
dsINMET = open('/content/drive/MyDrive/Datasets_TDE/INMET_SE_SP_A701_SAO PAULO - MIRANTE_01-01-2022_A_31-12-2022.CSV', encoding = 'iso-8859-1')
linhas = dsINMET.readlines()

#Obtenção do valor de latitude onde foi obtida o índice pluviométrico
latitudeINMET = linhas[4].split(';')[1]
latitudeINMET = latitudeINMET.replace(',', '.')
latitudeINMET = float(latitudeINMET[0: -2])

#Obtenção do valor de longitude onde foi obtida o índice pluviométrico
longitudeINMET = linhas[5].split(';')[1]
longitudeINMET = longitudeINMET.replace(',', '.')
longitudeINMET = float(longitudeINMET[0: -2])

#Definição da distância máxima de latitude e longitude de um acidente
DISTANCIA_LATITUDE_LONGITUDE = 0.05

#Leitura do dataset do datatran, obtendo as colunas "data_inversa", "municipio", "latitude", "longitude" utilizando o pandas
dfDatatran = pd.read_csv('/content/drive/MyDrive/Datasets_TDE/datatran2022.csv', encoding='utf-8', sep=';', decimal=",", usecols=['data_inversa', 'municipio', 'latitude', 'longitude'])

#Filtragem do dataframe pelos acidentes com valor "SAO PAULO" na coluna "municipio" e alteração do nome da coluna "data_inversa" para "Data"
dfDatatranFiltrada = dfDatatran.loc[dfDatatran['municipio'] == 'SAO PAULO']
dfDatatranFiltrada = dfDatatranFiltrada.rename(columns={'data_inversa' : 'Data'})

#Transformação da coluna latitude e longitude para valor numérico
dfDatatranFiltrada['latitude'] = pd.to_numeric(dfDatatranFiltrada['latitude'])
dfDatatranFiltrada['longitude'] = pd.to_numeric(dfDatatranFiltrada['longitude'])

#Filtragem pela latitude e longitude do acidente, se estiver fora do range da latitude e longitude pego no dataset INMET mais a distância máxima definida, a linha é descartada
dfDatatranFiltrada = dfDatatranFiltrada.loc[(dfDatatran['latitude'] >= latitudeINMET - DISTANCIA_LATITUDE_LONGITUDE) & (dfDatatran['latitude'] <= latitudeINMET + DISTANCIA_LATITUDE_LONGITUDE)]
dfDatatranFiltrada = dfDatatranFiltrada.loc[(dfDatatran['longitude'] >= longitudeINMET - DISTANCIA_LATITUDE_LONGITUDE) & (dfDatatran['longitude'] <= longitudeINMET + DISTANCIA_LATITUDE_LONGITUDE)]

#Alteração do nome da coluna "data_inversa" para "Data"
dfDatatranFiltrada = dfDatatranFiltrada.rename(columns={'data_inversa' : 'Data'})

#Remoção das colunas latitude e longitude e agrupamento das linhas pela data do acidente
dfDatatranAgrupada = dfDatatranFiltrada.drop(columns=['latitude', 'longitude'])
dfDatatranAgrupada = dfDatatranAgrupada.groupby('Data').count()

#Alteração do nome da coluna "municipio" para "Quantidade de acidentes"
dfDatatranAgrupada = dfDatatranAgrupada.rename(columns={'municipio' : 'Quantidade de acidentes'})

#Preenchimento das datas sem acidentes com o valor 0 entre o primeiro dia de 2022 e o último dia apresentado no dataset do datatran (nos dias que não tem acidentes não é inserida uma linha)
dataAnterior = dt.date(2022, 1, 1)
for index, row in dfDatatranAgrupada.iterrows():

  while dataAnterior < dt.datetime.strptime(index, '%Y-%m-%d').date():
    dfDatatranAgrupada.loc[str(dataAnterior)] = [0]
    dfDatatranAgrupada = dfDatatranAgrupada.sort_index()
    dataAnterior = dataAnterior + dt.timedelta(days=1)

  dataAnterior = dataAnterior + dt.timedelta(days=1)

#Preenchimento das datas sem acidentes com o valor 0 depois do último dia apresentado no dataset do datatran até o final do ano de 2022
while dataAnterior.year == 2022:
  dfDatatranAgrupada.loc[str(dataAnterior)] = [0]
  dataAnterior = dataAnterior + dt.timedelta(days=1)

#Transformação do index do dataframe (data dos acidente) em uma coluna
dfDatatranDiario = dfDatatranAgrupada.reset_index()

#Agrupamento das linhas pelo mês do acidente
dfDatatranMensal = dfDatatranDiario.groupby(dfDatatranDiario.Data.str[:7]).sum()

#Transformação do index do dataframe (mês dos acidente) em uma coluna e alteração do nome da coluna "Data" para "Mês"
dfDatatranMensalFormatada = dfDatatranMensal.reset_index()
dfDatatranMensalFormatada = dfDatatranMensalFormatada.rename(columns={'Data': 'Mês'})

#Abertura do dataset do INMET para obter as linhas do dataset sem as informações acima
dsINMET = open('/content/drive/MyDrive/Datasets_TDE/INMET_SE_SP_A701_SAO PAULO - MIRANTE_01-01-2022_A_31-12-2022.CSV', encoding = 'iso-8859-1')

#Criação de um arquivo .csv contendo apenas as informações das linhas
dsINMETAjustado = open('DatasetINMET.csv', 'w')
linhas = dsINMET.readlines()
dsINMETAjustado.writelines(linhas[8:-1])

#Ajuste de problemas no dataset do INMET (datas utilizando '/' ao invés de '-' e falta do 0 antes da vírgula para valores decimais)
dsINMETAjustado = open('DatasetINMET.csv', 'r')
dsINMETConteudo = dsINMETAjustado.read()
dsINMETConteudo = dsINMETConteudo.replace(';,', ';0,')
dsINMETConteudo = dsINMETConteudo.replace('/', '-')

#Escrita do dataset ajustado no .csv criado
dsINMETAjustado = open('DatasetINMET.csv', 'w')
dsINMETAjustado.write(dsINMETConteudo)

#Leitura do dataset do datatran, obtendo as colunas "Data", "PRECIPITAÇÃO TOTAL, HORÁRIO (mm)" utilizando o pandas
dfINMET = pd.read_csv('/content/DatasetINMET.csv', encoding='utf-8', sep=';', decimal=",", usecols=['Data', 'PRECIPITAÇÃO TOTAL, HORÁRIO (mm)'])

#Transformação da coluna "PRECIPITAÇÃO TOTAL, HORÁRIO (mm)" para valor numérico
dfINMET['PRECIPITAÇÃO TOTAL, HORÁRIO (mm)'] = pd.to_numeric(dfINMET['PRECIPITAÇÃO TOTAL, HORÁRIO (mm)'])

#Agrupamento das linhas pelo dia da precipitação total e alteração do nome da coluna "PRECIPITAÇÃO TOTAL, HORÁRIO (mm)" para "Precipitação Total"
dfINMETAgrupada = dfINMET.groupby(['Data']).sum()
dfINMETAgrupada = dfINMETAgrupada.rename(columns={'PRECIPITAÇÃO TOTAL, HORÁRIO (mm)' : 'Precipitação Total'})

#Transformação do index do dataframe (data da precipitação total) em uma coluna
dfINMETDiaria = dfINMETAgrupada.reset_index()

#Agrupamento das linhas pelo mês da precipitação total
dfINMETMensal = dfINMETDiaria.groupby(dfINMETDiaria.Data.str[:7]).sum()

#Transformação do index do dataframe (mês da precipitação total) em uma coluna e alteração do nome da coluna "Data" para "Mês"
dfINMETMensalFormatada = dfINMETMensal.reset_index()
dfINMETMensalFormatada = dfINMETMensalFormatada.rename(columns={'Data': 'Mês'})

#Remoção do arquivo .csv criado
os.remove('DatasetINMET.csv')

#Criação do dataframe de correlação
dfCorrelacao = pd.DataFrame()

#Criação da coluna "Quantidade de acidentes" e "Precipitação Total" obtendo os valores mensais dos dataframe do datatran e INMET
dfCorrelacao['Quantidade de acidentes']= dfDatatranMensal['Quantidade de acidentes']
dfCorrelacao['Precipitação Total'] = dfINMETMensal['Precipitação Total']

#Obtenção do valor total de acidentes e de precipitação total do ano
totalQtAcidentes = dfCorrelacao['Quantidade de acidentes'].sum()
totalPrecipTotal = dfCorrelacao['Precipitação Total'].sum()

#Criação das colunas de porcentagem utilizando o total do ano
dfCorrelacao['Quantidade de acidentes (%)'] = round(dfCorrelacao['Quantidade de acidentes'] / totalQtAcidentes * 100, 2)
dfCorrelacao['Precipitação Total (%)'] = round(dfCorrelacao['Precipitação Total'] / totalPrecipTotal * 100, 2)

#Transformação do index do dataframe (mês) em uma coluna e alteração do nome da coluna "Data" para "Mês"
dfCorrelacaoFormatada = dfCorrelacao.reset_index()
dfCorrelacaoFormatada = dfCorrelacaoFormatada.rename(columns={'Data': 'Mês'})

#Apresentação da tabela de "Porcentagem do Total da Quantidade de acidentes e da Precipitação no município de São Paulo" utilizando o plotly.express
fig = px.line(dfCorrelacaoFormatada, x='Mês', y=dfCorrelacao.columns[2:], title='Porcentagem do Total da Quantidade de acidentes e da Precipitação no município de São Paulo', labels={"value" : "Porcentagem", "variable" : "Legenda:"})
fig.show()

Mounted at /content/drive


<ipython-input-1-4d12fb1184c6>:51: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  dfDatatranMensal = dfDatatranDiario.groupby(dfDatatranDiario.Data.str[:7]).sum()
<ipython-input-1-4d12fb1184c6>:77: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  dfINMETMensal = dfINMETDiaria.groupby(dfINMETDiaria.Data.str[:7]).sum()
